<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/DC_OPF_Uncertainty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
#@title
!pip install -q gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def status(model):
  if model.Status==2:
    print('Status = optimal')
  elif model.Status==3:
    print('Status = infeasible')

     |████████████████████████████████| 11.5 MB 8.5 MB/s 


# Data

In [ ]:
#@title
# generation data
gen = pd.DataFrame({        
       'unit':      [0,    1,     2], 
       'bus':       [0,    1,     2],
       'cost':      [20,   25,    30], 
       'cost_up':   [40,   35,    35], 
       'cost_do':   [0,    15,    25], 
       'pmin':      [0,    0,     0], 
       'pmax':      [200,  150,   100]})

# line data
lin = pd.DataFrame({
       'line': [0,   1,   2], 
       'from': [0,   0,   1],
       'to':   [1,   2,   2], 
       'b':    [10,  10,  10], 
       'cap':  [25,  100, 100]})

# Numbers
ngen = len(gen)
nbus = max(max(lin['from']),max(lin['to']))+1
nlin = len(lin)

# cost_bus and pmax_bus vector
cost_bus = np.zeros(nbus)
cost_up_bus = np.zeros(nbus)
cost_do_bus = np.zeros(nbus)
pmax_bus = np.zeros(nbus)
for g in range(ngen):
  cost_bus[gen.loc[g,'bus']]=gen.loc[g,'cost']
  cost_up_bus[gen.loc[g,'bus']]=gen.loc[g,'cost_up']
  cost_do_bus[gen.loc[g,'bus']]=gen.loc[g,'cost_do']
  pmax_bus[gen.loc[g,'bus']]=gen.loc[g,'pmax']

# cap_line vector
cap_line = np.zeros(nlin)
for l in range(nlin):
  cap_line[lin.loc[l,'line']] = lin.loc[l,'cap']

# Power Transfer Distribution Factors
lin_bus = np.zeros((nlin,nbus))
for l in range(nlin):
  lin_bus[l,lin.loc[l,'from']-1]=1
  lin_bus[l,lin.loc[l,'to']-1]=-1    
matrixX_inv = np.diag(1/lin['b'])
matrixA = np.delete(np.array(lin_bus),0,axis=1)
matrixB = np.linalg.multi_dot([matrixA.T,matrixX_inv,matrixA]) 
ptdf = np.linalg.multi_dot([matrixX_inv,matrixA,np.linalg.inv(matrixB)])
ptdf = np.round(np.insert(ptdf,0,np.zeros((nlin)),axis=1),5)

# postive/negative ptdf
pos_neg = np.zeros((nlin,nbus))
for l in range(nlin):
  for b in range(nbus):
    if ptdf[l,b] > 0:
      pos_neg[l,b]=1
    elif ptdf[l,b] < 0:
      pos_neg[l,b]=-1

# scenario generation
nscen = 1000
demand = np.array([100,100,100])
var_10 = np.array([10,10,10])
var_20 = np.array([20,20,20])
var_25 = np.array([25,25,25])
var_30 = np.array([30,30,30])
scen_10 = 100 + 10*np.random.uniform(-1,1,size=(nscen,nbus))
scen_20 = 100 + 20*np.random.uniform(-1,1,size=(nscen,nbus))
scen_25 = 100 + 25*np.random.uniform(-1,1,size=(nscen,nbus))
scen_30 = 100 + 30*np.random.uniform(-1,1,size=(nscen,nbus))

# Deterministic DC-OPF

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n\\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(g_n-\hat{d}_n) \leq \overline{f}_l, \quad \forall l
\end{align}
$$

In [ ]:
#@title
def dc_opf_det(demand):
  # model
  m = gp.Model() 
  # variables
  gen = m.addMVar(nbus,name='gen')
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr( gen.sum() == demand.sum())
  m.addConstr(gen <= pmax_bus)
  m.addConstr(ptdf @ gen >= -cap_line + ptdf @ demand)
  m.addConstr(ptdf @ gen <= cap_line + ptdf @ demand)
  # solve
  m.write('det.lp')
  m.setParam('OutputFlag',0)
  m.optimize()
  status(m)
  # results
  print('cost =',m.ObjVal)
  print('dispatch =',gen.X)
  return gen.X
gen_det = dc_opf_det(demand)

Restricted license - for non-production use only - expires 2023-10-25
Status = optimal
cost = 7124.9962499625
dispatch = [175.00075001  24.99924999 100.        ]


# Real time operation

$$
\begin{align}
\underset{r^u_n,r^d_n}{\min} \quad & \sum_n c_n g^*_n + c^u_n r^u_n + c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n g^*_n+r^u_n-r^d_n = \sum_n \tilde{d}_n\\
& 0 \leq g^*_n + r^u_n-r^d_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(g^*_n + r^u_n-r^d_n -\tilde{d}_n) \leq \overline{f}_l, \quad \forall l \\
& r^u_n, r^d_n\geq 0
\end{align}
$$

In [ ]:
#@title
def real_time(demand,dispatch):
  # print('Solving real_time for demand=',demand)
  # model
  m = gp.Model()
  # variables
  #res = m.addMVar(nbus,lb=-GRB.INFINITY)
  res_up = m.addMVar(nbus)
  res_do = m.addMVar(nbus)
  # objective function
  m.setObjective(cost_bus @ dispatch + cost_up_bus @ res_up - cost_do_bus @ res_do, GRB.MINIMIZE)
  # constraints
  m.addConstr(dispatch.sum() + res_up.sum() - res_do.sum() == demand.sum())
  m.addConstr(dispatch + res_up - res_do >= 0)
  m.addConstr(dispatch + res_up - res_do <= pmax_bus)
  m.addConstr(ptdf @ dispatch + ptdf @ res_up - ptdf @ res_do >= -cap_line + ptdf @ demand)
  m.addConstr(ptdf @ dispatch + ptdf @ res_up - ptdf @ res_do <= cap_line + ptdf @ demand)
  #m.addConstr(res >= -res_do)
  #m.addConstr(res <= res_up)
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  # status(m)
  # results
  #print('reserves =',res.X)
  #print('reserves_up =',res_up.X)
  #print('reserves_do =',res_do.X)
  return m.ObjVal

In [ ]:
#@title
def evaluate_scenarios(dispatch,scen):
  v_cost = []
  for s in range(nscen):
    obj = real_time(scen[s,:],dispatch)
    v_cost.append(obj)    
  print('minimum cost =',np.min(v_cost))
  print('average cost =',np.mean(v_cost))
  print('maximum cost =',np.max(v_cost))
  #plt.figure(facecolor='white')
  #plt.hist(v_cost, 10, facecolor='blue', alpha=0.5)
  #plt.show()  

# Evaluate deterministic dispatch for var = 10

In [ ]:
#@title
evaluate_scenarios(gen_det,scen_10)

minimum cost = 6711.684140404908
average cost = 7296.887778717294
maximum cost = 8147.559518028637


# Evaluate deterministic dispatch for var = 20

In [ ]:
#@title
evaluate_scenarios(gen_det,scen_20)

minimum cost = 6369.27481135716
average cost = 7438.3289715112005
maximum cost = 9195.245463676118


# Evaluate deterministic dispatch for var = 30

In [ ]:
#@title
evaluate_scenarios(gen_det,scen_30)

minimum cost = 5983.6354867980235
average cost = 7673.956507380126
maximum cost = 10187.588612163796


# Robust (no recourse)

$d_n = \hat{d}_n + \overline{e}_n \eta_n$ with $\eta_n \in [-1,1]$

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & \sum_n g_n \geq \underset{\eta_n\in[-1,1]}{\max} \sum_n \hat{d}_n + \overline{e}_n \eta_n \\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n -\hat{d}_n - \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n -\hat{d}_n - \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l 
\end{align}
$$

Reformulation 

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & \sum_n g_n \geq \sum_n \hat{d}_n + \overline{e}_n \\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n -\hat{d}_n - \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n -\hat{d}_n + \overline{e}_n) , \quad \forall l \\
& \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n -\hat{d}_n + \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n -\hat{d}_n - \overline{e}_n) \leq \overline{f}_l, \quad \forall l \\
\end{align}
$$

where $\mathcal{N}^+_l$ set of nodes such that $b_{ln}>0$ and $\mathcal{N}^-_l$ set of nodes such that $b_{ln}\lt 0$ 

In [ ]:
#@title
def dc_opf_robust(demand,var):
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus,name='gen')
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen.sum() >= demand.sum() + var.sum())
  m.addConstr(gen <= pmax_bus)
  m.addConstr(ptdf @ gen >= -cap_line + ptdf @ demand + np.multiply(ptdf,pos_neg) @ var)
  m.addConstr(ptdf @ gen <= cap_line + ptdf @ demand - np.multiply(ptdf,pos_neg) @ var)
  # solve
  m.setParam('OutputFlag',0)
  m.write('robust.lp')
  m.optimize()
  status(m)
  # results
  print('dispatch =',gen.X)
  return gen.X

# Evaluate robust dispatch for var = 10

In [ ]:
#@title
gen_robust = dc_opf_robust(demand,var_10)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [175.00045  54.99955 100.     ]
minimum cost = 7011.682640479907
average cost = 7440.943111649432
maximum cost = 7863.234650532272


# Evaluate robust dispatch for var = 20

In [ ]:
#@title
gen_robust = dc_opf_robust(demand,var_20)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [175.00015  84.99985 100.     ]
minimum cost = 6969.268811297157
average cost = 7744.984822920886
maximum cost = 8612.23991292262


# Evaluate robust dispatch for var = 30

In [ ]:
#@title
#gen_robust = dc_opf_robust(demand,var_30)
#evaluate_scenarios(gen_robust,scen_30)

If we consider a budget $\Gamma$ we have to use the duals

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & \sum_n \overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \alpha \\
& \eta^+_n \leq 1: \beta^+_n, \quad \forall n\\
& \eta^-_n \leq 1: \beta^-_n, \quad \forall n \\
& \eta^+_n,\eta^-_n\geq 0 
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\alpha, \beta^+_n,\beta^-_n}{\min} \quad & \alpha\Gamma + \sum_n  \beta^+_n+\beta^-_n  \\
\text{s.t.} \quad & -\overline{e}_n + \alpha + \beta^+_n \geq 0, \quad \forall n \\
& \overline{e}_n + \alpha + \beta^-_n \geq 0, \quad \forall n \\
& \alpha, \beta^+_n,\beta^-_n\geq 0
\end{align}
$$

With the other constraint

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\min} \quad & \sum_n -b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \gamma_l \\
& \eta^+_n \leq 1: \delta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \delta^-_{ln}, \quad \forall n \\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\gamma_l, \delta^+_{ln},\delta^-_{ln}}{\max} \quad & -\gamma_l\Gamma - \sum_n  \delta^+_{ln}+\delta^-_{ln}  \\
\text{s.t.} \quad & -b_{ln}\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n \\
& \gamma_l, \delta^+_{ln},\delta^-_{ln}\geq 0
\end{align}
$$

Last constraint

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & \sum_n -b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \epsilon_l \\
& \eta^+_n \leq 1: \zeta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \zeta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad & -\epsilon_l\Gamma - \sum_n  \zeta^+_{ln}+\zeta^-_{ln}  \\
\text{s.t.} \quad & -b_{ln}\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n \\
& \epsilon_l, \zeta^+_{ln},\zeta^-_{ln}\geq 0
\end{align}
$$

We replace everything and obtain

$$
\begin{align}
\underset{g_n,\alpha,\beta^+_n,\beta^-_n,\gamma_l,\delta^+_{ln},\delta^-_{ln},\epsilon_l,\zeta^+_{ln},\zeta^-_{ln}}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& \sum_n g_n \geq \alpha\Gamma + \sum_n  \hat{d}_n + \beta^+_n+\beta^-_n \\
& -\overline{f}_l \leq -\gamma_l\Gamma - \sum_n \delta^+_{ln}+\delta^-_{ln} - b_{ln}(g_n-\hat{d}_n), \quad \forall l \\
&  -\epsilon_l\Gamma - \sum_n  \zeta^+_{ln}+\zeta^-_{ln}-b_{ln}(g_n-\hat{d}_n) \leq \overline{f}_l, \quad \forall l \\
& -\overline{e}_n + \alpha + \beta^+_n \geq 0, \quad \forall n \\
& \overline{e}_n + \alpha + \beta^-_n \geq 0, \quad \forall n \\
& -b_{ln}\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall l,n \\
& b_{ln}\overline{e}_n + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall l,n \\
& -b_{ln}\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall l,n \\
& b_{ln}\overline{e}_n + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall l,n \\
& \alpha,\beta^+_n,\beta^-_n,\gamma_l,\delta^+_{ln},\delta^-_{ln},\epsilon_l,\zeta^+_{ln},\zeta^-_{ln} \geq 0
\end{align}
$$

In [ ]:
#@title
def dc_opf_robust_budget(demand,var,budget):
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus)
  alpha = m.addVar()
  beta_up = m.addMVar(nbus)
  beta_do = m.addMVar(nbus)
  gamma = m.addMVar(nlin)
  delta_up = m.addMVar((nlin,nbus))
  delta_do = m.addMVar((nlin,nbus))
  epsilon = m.addMVar(nlin)
  zeta_up = m.addMVar((nlin,nbus))
  zeta_do = m.addMVar((nlin,nbus))
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen <= pmax_bus)
  m.addConstr(gen.sum() >= alpha*budget + demand.sum() + beta_up.sum() + beta_do.sum() )  
  m.addConstrs(-cap_line[l] <= - budget*gamma[l] - delta_up[l,:].sum() - delta_do[l,:].sum() + ptdf[l,:] @ gen - ptdf[l,:] @ demand for l in range(nlin))
  m.addConstrs(-budget*epsilon[l] - zeta_up[l,:].sum() - zeta_do[l,:].sum() + ptdf[l,:] @ gen - ptdf[l,:] @ demand <= cap_line[l] for l in range(nlin))
  m.addConstrs(-var[n] + alpha + beta_up[n] >= 0 for n in range(nbus))
  m.addConstrs(var[n] + alpha + beta_do[n] >= 0 for n in range(nbus))
  m.addConstrs(-var[b]*ptdf[:,b] + gamma + delta_up[:,b] >= 0 for b in range(nbus))
  m.addConstrs(var[b]*ptdf[:,b] + gamma + delta_do[:,b] >= 0 for b in range(nbus))
  m.addConstrs(-var[b]*ptdf[:,b] + epsilon + zeta_up[:,b] >= 0 for b in range(nbus))
  m.addConstrs(var[b]*ptdf[:,b] + epsilon + zeta_do[:,b] >= 0 for b in range(nbus))
  # solve
  m.setParam('OutputFlag',0)
  m.write('robust_ldr.lp')
  m.optimize()
  status(m)
  # results
  print('dispatch =',gen.X)
  return gen.X

# Evaluate robust dispatch (Gamma=2, var=10)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_10,2)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [165.00045  54.99955 100.     ]
minimum cost = 6861.685640419908
average cost = 7295.302803884724
maximum cost = 7966.979865795695


# Evaluate robust dispatch (Gamma=2, var=20)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_20,2)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [155.00015  84.99985 100.     ]
minimum cost = 6597.246511248685
average cost = 7453.1699082678515
maximum cost = 8610.339745249385


# Evaluate robust dispatch (Gamma=2, var=30)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_30,2)
evaluate_scenarios(gen_robust,scen_30)

Status = optimal
dispatch = [144.99985 115.00015 100.     ]
minimum cost = 6412.512336522224
average cost = 7646.824258149024
maximum cost = 9559.24795348586


# Evaluate robust dispatch (Gamma=1, var=10)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_10,1)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [165.00045  44.99955 100.     ]
minimum cost = 6761.685640419908
average cost = 7241.97596163932
maximum cost = 8066.979865795695


# Evaluate robust dispatch (Gamma=1, var=20)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_20,1)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [155.00015  64.99985 100.     ]
minimum cost = 6397.246511248686
average cost = 7346.571007370666
maximum cost = 8810.339745249386


# Evaluate robust dispatch (Gamma=1, var=30)

In [ ]:
#@title
gen_robust = dc_opf_robust_budget(demand,var_30,1)
evaluate_scenarios(gen_robust,scen_30)

Status = optimal
dispatch = [144.99985  85.00015 100.     ]
minimum cost = 6112.512336522224
average cost = 7494.11343446798
maximum cost = 9859.247953485861


# Linear decision rules

We define $\Omega = \sum_n \overline{e}_n\eta_n$, $\underline{\Omega}=-\sum_n\overline{e}_n$, and $\overline{\Omega}=\sum_n\overline{e}_n$. We use a linear decision rule so that the output of each unit is determined by $g_n-\beta_n\Omega$. We first minimize the expected cost

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad & \mathbb{E} \left[ \sum_n c_n (g_n-\beta_n \Omega) \right] + c^u_n r^u_n + c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n g_n-\beta_n\Omega = \sum_n \hat{d}_n + \overline{e}_n\eta_n \\
& 0 \leq g_n - \beta_n\Omega \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n - \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n - \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l \\
& -r^d_n \leq \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\min} -\beta_n \Omega, \quad \forall n \\
& \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\max} -\beta_n \Omega \leq r^u_n, \quad \forall n \\
& \beta_n \geq 0
\end{align}
$$

We can reformulate as 

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad &  \sum_n c_n g_n + c^u_n r^u_n + c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\max}\beta_n\Omega, \quad \forall n\\
& g_n - \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\min}\beta_n\Omega \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n - \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n - \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l \\
& -r^d_n \leq \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\min} -\beta_n \Omega, \quad \forall n \\
& \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\max} -\beta_n \Omega \leq r^u_n, \quad \forall n \\
& \beta_n \geq 0
\end{align}
$$

And therefore

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad &  \sum_n c_n g_n + c^u_n r^u_n + c^d_n r^d_n  \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \beta_n\overline{\Omega}, \quad \forall n\\
& g_n - \beta_n\underline{\Omega} \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n - \beta_n\overline{\Omega} -\hat{d}_n - \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n - \beta_n\underline{\Omega} -\hat{d}_n + \overline{e}_n), \quad \forall l \\
&  \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n - \beta_n\underline{\Omega} -\hat{d}_n + \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n - \beta_n\overline{\Omega} -\hat{d}_n - \overline{e}_n ) \leq \overline{f}_l, \quad \forall l \\
& -r^d_n \leq -\beta_n \overline{\Omega}, \quad \forall n \\
& -\beta_n \underline{\Omega} \leq r^u_n, \quad \forall n \\
& \beta_n \geq 0
\end{align}
$$

In [ ]:
#@title
def dc_opf_ldr(demand,var):
  # model
  m = gp.Model()
  # variables
  gen = m.addMVar(nbus,name='gen')
  #res_up = m.addMVar(nbus,name='res_up')
  #res_do = m.addMVar(nbus,name='res_do')
  beta = m.addMVar(nbus,name='beta')
  # objective function
  #m.setObjective(cost_bus @ gen + cost_up_bus @ res_up + cost_do_bus @ res_do, GRB.MINIMIZE)
  m.setObjective(cost_bus @ gen + 0.25 * var.sum() * cost_up_bus @ beta + 0.25 * var.sum() * cost_do_bus @ beta, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen.sum() == demand.sum())
  m.addConstr(beta.sum() == 1)
  m.addConstr(0 <= gen - var.sum()*beta)
  m.addConstr(gen + var.sum()*beta <= pmax_bus)
  m.addConstr(ptdf @ gen - var.sum() * ptdf @ beta >= -cap_line + ptdf @ demand + np.multiply(ptdf,pos_neg) @ var)
  m.addConstr(ptdf @ gen - var.sum() * ptdf @ beta <= cap_line + ptdf @ demand - np.multiply(ptdf,pos_neg) @ var)
  #m.addConstr(-res_do <= -var.sum()*beta)
  #m.addConstr(var.sum()*beta <= res_up)
  # solve
  m.setParam('OutputFlag',0)
  m.write('ldr.lp')
  m.optimize()
  status(m) 
  # results
  print('dispatch =',gen.X,beta.X)
  return gen.X

# Evaluate LDR dispatch with var = 10

In [ ]:
#@title
gen_robust_ldr = dc_opf_ldr(demand,var_10)
evaluate_scenarios(gen_robust_ldr,scen_10)

Status = optimal
dispatch = [145.00045  54.99955 100.     ] [1. 0. 0.]
minimum cost = 6617.854936685488
average cost = 7316.832793348991
maximum cost = 8266.979865795696


# Evaluate LDR dispatch with var = 20

In [ ]:
#@title
gen_robust_ldr = dc_opf_ldr(demand,var_20)
evaluate_scenarios(gen_robust_ldr,scen_20)

Status = optimal
dispatch = [115.00015  84.99985 100.     ] [1. 0. 0.]
minimum cost = 6059.150418431221
average cost = 7497.745360044772
maximum cost = 9210.339745249385


# Evaluate LDR dispatch with var = 30


In [ ]:
#@title
#gen_robust_ldr = dc_opf_ldr(demand,var_30)
#evaluate_scenarios(gen_robust_ldr,scen_30)

First primal

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & \beta_n\sum_m \overline{e}_m(\eta^+_m-\eta^-_m)  \\
\text{s.t.} \quad & \sum_m \eta^+_m+\eta^-_m \leq \Gamma: \tau \\
& \eta^+_m \leq 1: \rho^+_{m}, \quad \forall m\\
& \eta^-_m \leq 1: \rho^-_{m}, \quad \forall m\\
& \eta^+_m,\eta^-_m\geq 0
\end{align}
$$


First dual

$$
\begin{align}
\underset{\tau,\rho^+_m,\rho^-_m}{\min} \quad & \Gamma\tau + \sum_m \rho^+_m + \rho^-_m  \\
\text{s.t.} \quad & -\beta_n\overline{e}_m + \tau + \rho^+_m \geq 0, \quad \forall m \\
& \beta_n\overline{e}_m + \tau + \rho^-_m \geq 0, \quad \forall m\\
& \tau,\rho^+_m,\rho^-_m \geq 0
\end{align}
$$

Second primal

$$
\begin{align}
\underset{\eta^+_m,\eta^-_m}{\min} \quad & \beta_n \sum_m \overline{e}_m (\eta^+_m-\eta^-_m)  \\
\text{s.t.} \quad & \sum_m \eta^+_m+\eta^-_m \leq \Gamma: \kappa \\
& \eta^+_m \leq 1: \mu^+_{m}, \quad \forall m\\
& \eta^-_m \leq 1: \mu^-_{m}, \quad \forall m\\
& \eta^+_m,\eta^-_m\geq 0
\end{align}
$$

Second dual

$$
\begin{align}
\underset{\kappa,\mu^+_n,\mu^-_n}{\max} \quad & -\Gamma\kappa - \sum_m \mu^+_m + \mu^-_m  \\
\text{s.t.} \quad & \beta_n\overline{e}_m + \kappa + \mu^+_m \geq 0, \quad \forall m \\
& -\beta_n\overline{e}_m + \kappa + \mu^-_m \geq 0, \quad \forall m\\
& \kappa,\mu^+_m,\mu^-_m \geq 0
\end{align}
$$

Third primal

$$
\begin{align}
\underset{\Omega,\eta^+_n,\eta^-_n}{\min} \quad & \sum_n -b_{ln}\beta_n\Omega -b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \Omega = \sum_n \overline{e}_n (\eta^+_n-\eta^-_n): \alpha_l \\
& \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \gamma_l \\
& \eta^+_n \leq 1: \delta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \delta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

Third dual

$$
\begin{align}
\underset{\alpha_l, \gamma_l, \delta^+_{ln},\delta^-_{ln}}{\max} \quad & - \gamma_l\Gamma - \sum_n  \delta^+_{ln}+\delta^-_{ln}  \\
\text{s.t.} \quad & \sum_n b_{ln}\beta_n - \alpha_l = 0 \\
& -b_{ln}\overline{e}_n + \alpha_l\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n - \alpha_l \overline{e}_n+ \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n \\
& \gamma_l, \delta^+_{ln},\delta^-_{ln}\geq 0
\end{align}
$$

Four primal

$$
\begin{align}
\underset{\Omega,\eta^+_n,\eta^-_n}{\max} \quad & \sum_n -b_{ln}\beta_n\Omega -b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \Omega = \sum_n \overline{e}_n (\eta^+_n-\eta^-_n): \lambda_l \\
& \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \epsilon_l \\
& \eta^+_n \leq 1: \zeta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \zeta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

Four dual

$$
\begin{align}
\underset{\lambda_l, \epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad & - \epsilon_l\Gamma - \sum_n  \zeta^+_{ln}+\zeta^-_{ln}  \\
\text{s.t.} \quad & \sum_n b_{ln}\beta_n - \lambda_l = 0 \\
& -b_{ln}\overline{e}_n + \lambda_l\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n - \lambda_l \overline{e}_n+ \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n \\
& \epsilon_l, \zeta^+_{ln},\zeta^-_{ln}\geq 0
\end{align}
$$

By using the dual formulation we obtain

$$
\begin{align}
\underset{g_n,\beta_n,\tau,\rho^+_n,\rho^-_n,\kappa,\mu^+_n,\mu^-_n,\alpha_l,\gamma_l, \delta^+_{ln},\delta^-_{ln}, \lambda_l,\epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad &  \sum_n c_n g_n \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \Gamma\tau - \sum_n \rho^+_n - \rho^-_n, \quad \forall n\\
& -\beta_n\overline{e}_m + \tau + \rho^+_m \geq 0, \quad \forall n,m \\
& \beta_n\overline{e}_m + \tau + \rho^-_m \geq 0, \quad \forall n,m\\
& g_n + \Gamma\kappa + \sum_n \mu^+_n + \mu^-_n \leq \overline{g}_n, \quad \forall n\\
& \beta_n\overline{e}_m + \kappa + \mu^+_m \geq 0, \quad \forall n,m \\
& -\beta_n\overline{e}_m + \kappa + \mu^-_m \geq 0, \quad \forall n,m\\
& -\overline{f}_l \leq -\gamma_l\Gamma - \sum_n \delta^+_{ln} + \delta^-_{ln}, \quad \forall l \\
& \sum_n b_{ln}\beta_n - \alpha_l = 0, \quad \forall l \\
& -b_{ln}\overline{e}_n + \alpha_l\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n,l \\
& b_{ln}\overline{e}_n - \alpha_l \overline{e}_n+ \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n,l \\
&  -\epsilon_l\Gamma - \sum_n \zeta^+_{ln}+\zeta^-_{ln} \leq \overline{f}_l, \quad \forall l \\
& \sum_n b_{ln}\beta_n - \lambda_l = 0, \quad \forall l \\
& -b_{ln}\overline{e}_n + \lambda_l\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n,l \\
& b_{ln}\overline{e}_n - \lambda_l \overline{e}_n+ \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n,l \\
& \beta_n, \tau,\rho^+_n,\rho^-_n, \kappa,\mu^+_n,\mu^-_n,\gamma_l, \delta^+_{ln},\delta^-_{ln}, \epsilon_l, \zeta^+_{ln},\zeta^-_{ln} \geq 0
\end{align}
$$

In [ ]:
def dc_opf_ldr_budget(demand,var,budget):
  # model
  m = gp.Model()
  m.Params.Threads = 1
  # variables
  gen = m.addMVar(nbus,name='gen')
  beta = m.addMVar(nbus,name='beta')
  tau = m.addVar(name='tau')
  rho_up = m.addMVar(nbus,name='rho_up')
  rho_do = m.addMVar(nbus,name='rho_do')
  kappa = m.addVar(name='kappa')
  mu_up = m.addMVar(nbus,name='mu_up')
  mu_do = m.addMVar(nbus,name='mu_do')
  alpha = m.addMVar(nlin,lb=-GRB.INFINITY,name='alpha')
  gamma = m.addMVar(nlin,name='gamma')  
  delta_up = m.addMVar((nlin,nbus),name='delta_up')
  delta_do = m.addMVar((nlin,nbus),name='delta_do')
  lamda = m.addMVar(nlin,lb=-GRB.INFINITY,name='lambda')
  epsilon = m.addMVar(nlin,name='epsilon')
  zeta_up = m.addMVar((nlin,nbus),name='zeta_up')
  zeta_do = m.addMVar((nlin,nbus),name='zeta_do')
  # objective function
  m.setObjective(cost_bus @ gen + 0.5 * var.sum() * var.sum() * cost_up_bus @ beta + 0.5 * var.sum() * var.sum() * cost_do_bus @ beta, GRB.MINIMIZE)
  #m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen.sum() == demand.sum())
  m.addConstr(beta.sum() == 1)
  m.addConstrs(0 <= gen[n] - budget*tau - rho_up.sum() - rho_do.sum() for n in range(nbus))
  m.addConstrs(-beta[n]*var[m] + tau + rho_up[m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(beta[n]*var[m] + tau + rho_do[m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(gen[n] + budget*kappa + mu_up.sum() + mu_do.sum() <= pmax_bus[n] for n in range(nbus))
  m.addConstrs(beta[n]*var[m] + kappa + mu_up[m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(-beta[n]*var[m] + kappa + mu_do[m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(-cap_line[l] <= -budget*gamma[l] - delta_up[l,:].sum() - delta_do[l,:].sum() for l in range(nlin))
  m.addConstr(ptdf @ beta - alpha == 0)
  m.addConstrs(-ptdf[l,n]*var[n] + alpha[l]*var[n] + gamma[l] + delta_up[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(ptdf[l,n]*var[n] - alpha[l]*var[n] + gamma[l] + delta_do[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(-budget*epsilon[l] - zeta_up[l,:].sum() - zeta_do[l,:].sum() <= cap_line[l] for l in range(nlin))
  m.addConstr(ptdf @ beta - lamda == 0)
  m.addConstrs(-ptdf[l,n]*var[n] + lamda[l]*var[n] + epsilon[l] + zeta_up[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(ptdf[l,n]*var[n] - lamda[l]*var[n] + epsilon[l] + zeta_do[l,n] >= 0 for n in range(nbus) for l in range(nlin))  
  # solve
  m.setParam('OutputFlag',0)  
  m.optimize()
  status(m)   
  m.write('ldr_budget.lp')
  # results
  print('dispatch =',gen.X,beta.X)
  print('kappa=',kappa.X)
  print('mu_up=',mu_up.X)
  print('mu_do=',mu_do.X)
  return gen.X

# Evaluate LDR dispatch (Gamma=2, var=10)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_10,3)
evaluate_scenarios(gen_ldr_budget,scen_10)

Set parameter Threads to value 1
Status = optimal
dispatch = [170. 100.  30.] [1. 0. 0.]
kappa= 0.0
mu_up= [0. 0. 0.]
mu_do= [10. 10. 10.]
minimum cost = 7434.164886861534
average cost = 7936.277854814781
maximum cost = 8448.780527314091


# Evaluate LDR dispatch (Gamma=2, var=20)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_20,2)
evaluate_scenarios(gen_ldr_budget,scen_20)

Set parameter Threads to value 1
Status = optimal
dispatch = [180. 100.  20.] [1. 0. 0.]
minimum cost = 7203.360949349592
average cost = 8170.2376477602775
maximum cost = 9222.615642532528


# Evaluate LDR dispatch (Gamma=2, var=30)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_30,2)
evaluate_scenarios(gen_ldr_budget,scen_30)

Set parameter Threads to value 1
Status = optimal
dispatch = [172.5 100.   27.5] [0.83333333 0.08333333 0.08333333]
minimum cost = 6530.3724145596925
average cost = 8025.5189891839755
maximum cost = 9750.066111938793


# Evaluate LDR dispatch (Gamma=1, var=10)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_10,1)
evaluate_scenarios(gen_ldr_budget,scen_10)

Set parameter Threads to value 1
Status = optimal
dispatch = [190. 100.  10.] [1. 0. 0.]
minimum cost = 7934.164886861534
average cost = 8436.277854814782
maximum cost = 8948.780527314091


# Evaluate LDR dispatch (Gamma=1, var=20)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_20,1)
evaluate_scenarios(gen_ldr_budget,scen_20)

Set parameter Threads to value 1
Status = optimal
dispatch = [180. 100.  20.] [1. 0. 0.]
minimum cost = 7203.360949349592
average cost = 8170.2376477602775
maximum cost = 9222.615642532528


# Evaluate LDR dispatch (Gamma=1, var=30)

In [ ]:
#@title
gen_ldr_budget = dc_opf_ldr_budget(demand,var_30,1)
evaluate_scenarios(gen_ldr_budget,scen_30)

Set parameter Threads to value 1
Status = optimal
dispatch = [170. 100.  30.] [1. 0. 0.]
minimum cost = 6480.371289548442
average cost = 7966.334846547034
maximum cost = 9687.566111938793


# Stochastic programming

We use the index $w$ for scenarios and $\pi_w$ for scenario probability. First stage decision are $g_n$ while second stage decisions are $r_{nw}$. The two-stage optimization problem is formulated as follows:

$$
\begin{align}
\underset{g_n,r_{nw},r^u_n, r^d_n}{\min} \quad &  \sum_w \pi_w\left( \sum_n c_n (g_n+r_{nw}) + c^u_n r^u_n + c^d_n r^d_n\right)  \\
\text{s.t.} \quad & \sum_n g_n+r_{nw} = \sum_n \hat{d}_n + \overline{e}_n\eta_{nw}, \quad \forall w \\
& 0 \leq g_n + r_{nw} \leq \overline{g}_n, \quad \forall n,w\\
& -\overline{f}_l \leq \sum_n b_{ln}(g_n +r_{nw} -\hat{d}_n - \overline{e}_n \eta_{nw}) \leq \overline{f}_l, \quad \forall l,w \\
& -r^d_n \leq r_{nw} \leq r^u_n, \quad \forall n,w \\
& r^u_n, r^d_n\geq 0
\end{align}
$$

In [ ]:
#@title
def sto(demand,var,nscen):
  # generate scenarios
  eta = np.zeros((nbus,nscen))
  for n in range(nbus):
    for s in range(nscen):
      eta[n,s] = var[n]*np.random.uniform(-1,1)
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus)
  #res = m.addMVar((nbus,nscen),lb=-GRB.INFINITY)
  res_up = m.addMVar((nbus,nscen))
  res_do = m.addMVar((nbus,nscen))
  # objective function
  m.setObjective(cost_bus @ gen + sum(cost_up_bus @ res_up[:,s] for s in range(nscen))/nscen - sum(cost_do_bus @ res_do[:,s] for s in range(nscen))/nscen, GRB.MINIMIZE)
  # constraints
  m.addConstrs(gen.sum() + res_up[:,s].sum() - res_do[:,s].sum() == demand.sum() + eta[:,s].sum() for s in range(nscen))
  m.addConstrs(gen + res_up[:,s] - res_do[:,s] >= 0 for s in range(nscen))
  m.addConstrs(gen + res_up[:,s] - res_do[:,s] <= pmax_bus for s in range(nscen))
  m.addConstrs(ptdf @ gen + ptdf @ res_up[:,s] - ptdf @ res_do[:,s] >= -cap_line + ptdf @ demand + ptdf @ eta[:,s]  for s in range(nscen))
  m.addConstrs(ptdf @ gen + ptdf @ res_up[:,s] - ptdf @ res_do[:,s] <= cap_line + ptdf @ demand + ptdf @ eta[:,s] for s in range(nscen))
  #m.addConstrs(res[:,s] >= -res_do for s in range(nscen))
  #m.addConstrs(res[:,s] <= res_up for s in range(nscen))
  # solve
  m.setParam('OutputFlag',0)  
  m.optimize()
  status(m)     
  # results
  print('dispatch =',gen.X)  
  return gen.X

# Evaluate stochastic dispatch with var=10

In [ ]:
#@title
gen_sto = sto(demand,var_10,100)
evaluate_scenarios(gen_sto,scen_10)

Status = optimal
dispatch = [162.18539285  44.51136314  94.91935967]
minimum cost = 6689.1755574607605
average cost = 7230.122765325561
maximum cost = 8139.490793255101


# Evaluate stochastic dispatch with var=20

In [ ]:
#@title
gen_sto = sto(demand,var_20,100)
evaluate_scenarios(gen_sto,scen_20)

Status = optimal
dispatch = [153.83876068  59.41710786  91.30535274]
minimum cost = 6280.525362108894
average cost = 7324.3678506458045
maximum cost = 8927.061242799009


# Evaluate stochastic dispatch with var=30

In [ ]:
#@title
gen_sto = sto(demand,var_30,100)
evaluate_scenarios(gen_sto,scen_30)

Status = optimal
dispatch = [130.35653804 103.04585573  75.50725923]
minimum cost = 5950.86040343634
average cost = 7457.936657505991
maximum cost = 10031.156904072644
